In [1]:
import torch
import torchvision
import torchvision.transforms as transforms
import torchvision.models as models
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import time
from itertools import count
import natsort
import datetime
import numpy as np
import os
import math

In [2]:
from torch.utils.data import Dataset, DataLoader, WeightedRandomSampler
import albumentations as A
from albumentations.pytorch import ToTensorV2
import cv2
import glob
import numpy
import random
import pandas as pd
import tqdm
torch.manual_seed(10)

In [3]:
print(f"Is CUDA supported by this system? {torch.cuda.is_available()}")
print(f"CUDA version: {torch.version.cuda}")
# Storing ID of current CUDA device
cuda_id = torch.cuda.current_device()
print(f"ID of current CUDA device: {torch.cuda.current_device()}")
print(f"Name of current CUDA device: {torch.cuda.get_device_name(cuda_id)}")

device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print(device)

Is CUDA supported by this system? True
CUDA version: 11.3
ID of current CUDA device: 0
Name of current CUDA device: NVIDIA GeForce RTX 2070 Super
cuda:0


# Building the dataset

In [4]:
class SurgicalDataset(Dataset):
    def __init__(self, image_paths, labels, transform=False):
        super(SurgicalDataset, self).__init__()
        self.image_paths = image_paths
        self.labels = labels    #.astype(dtype='int')
        self.transform = transform
        
    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        image_filepath = self.image_paths[idx]
        image = cv2.imread(image_filepath)
        label = self.labels[idx]
        if self.transform is not None:
            image = self.transform(image=image)["image"]
        
        return image, label



In [5]:
def get_transform(model_name):

    if model_name == 'alexnet':
        transform = A.Compose([
            A.Resize(227, 227),
            A.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
            ToTensorV2(),
        ])

    else:

        transform = A.Compose([
            A.Resize(224,224),
            A.Normalize((0.5,0.5,0.5),(0.5,0.5,0.5)),
            ToTensorV2(),
        ])
    
    return transform

In [6]:
# Preparing the datasets
# Get images
train_image_paths = []
train_data_path = r"C:\Users\panji\EECS6691_Advanced_DL\Assignment2\training_data_images"
train_image_paths.append(glob.glob(train_data_path + '/*'))
# unpack the listed list
train_image_paths1 = [item for sublist in train_image_paths for item in sublist]
train_image_paths1 = natsort.natsorted(train_image_paths1)
print('len(train_image_paths1)', len(train_image_paths1))

# Get labels
df = pd.read_csv("Processed_data.csv")
df1 = df.loc[:,"Phases"].to_numpy()
df2 = df1.tolist()
print('len(df2)', len(df2))

# Preparing the datasets (images and labels)
dataset_train = pd.DataFrame(
    {'Link': train_image_paths1,
     'Label': df2,
    })
dataset_train1 = dataset_train.sample(frac=1, random_state=1)
train_image_paths = dataset_train1.loc[:,"Link"].to_numpy().tolist()
labels = dataset_train1.loc[:,"Label"].to_numpy().tolist()

# manually split the dataset
train_image_paths, valid_image_paths = train_image_paths[:int(0.8*len(train_image_paths))], train_image_paths[int(0.8*len(train_image_paths)):]  
train_labels, valid_labels = labels[:int(0.8*len(labels))], labels[int(0.8*len(labels)):] 
print('train_labels', len(train_labels))
print('train_image_paths', len(train_image_paths))
print('label distribution in the training data', np.bincount(train_labels))

len(train_image_paths1) 215057
len(df2) 215057
train_labels 172045
train_image_paths 172045
label distribution in the training data [  243  8681 22901 41140   952 22305   666 10930   896  2308 44928 12987
  1789  1246    73]


# Weighted Data Sampler

In [7]:
# from torch.utils.data import WeightedRandomSampler

# Get labels
df = pd.read_csv("Processed_data.csv")
df1 = df.loc[:,"Phases"].to_numpy()
df2 = df1.tolist()
print('len(df2)', len(df2))

# Preparing the datasets (images and labels)
dataset_train = pd.DataFrame(
    {'Link': train_image_paths1,
     'Label': df2,
    })
dataset_train1 = dataset_train.sample(frac=1, random_state=1)
train_image_paths = dataset_train1.loc[:,"Link"].to_numpy().tolist()
labels = dataset_train1.loc[:,"Label"].to_numpy().tolist()

summary = {i:0 for i in range(15)}
num_classes = 15
total_samples = 0
for i in train_labels:
    total_samples += 1
    summary[i] += 1
print(summary)
print(total_samples)

class_weights = [total_samples/summary[i] for i in range(num_classes)]
weights = [class_weights[train_labels[i]] for i in range(total_samples)]
sampler = WeightedRandomSampler(torch.DoubleTensor(weights), len(weights))
print(len(class_weights))
print(len(weights))
print(len(list(sampler)))

len(df2) 215057
{0: 243, 1: 8681, 2: 22901, 3: 41140, 4: 952, 5: 22305, 6: 666, 7: 10930, 8: 896, 9: 2308, 10: 44928, 11: 12987, 12: 1789, 13: 1246, 14: 73}
172045
15
172045
172045


# Building the classifier class

In [8]:
class Classifier():

    def __init__(self, name, model, dataloaders, parameter, use_cuda=False):
        
        '''
        @name: Experiment name. Will define stored results etc. 
        @model: Any models
        @dataloaders: Dictionary with keys train, val and test and corresponding dataloaders
        @class_names: list of classes, where the idx of class name corresponds to the label used for it in the data
        @use_cuda: whether or not to use cuda
        '''
       
        self.name = name
        if use_cuda and not torch.cuda.is_available():
            raise Exception("Asked for CUDA but GPU not found")
            
        self.use_cuda = use_cuda
        self.epoch = parameter['epochs']
        self.lr = parameter['lr']
        self.batch_size = parameter['batch_size']
        
        self.model = model.to('cuda' if use_cuda else 'cpu') # model.to('cpu')
        self.criterion = nn.CrossEntropyLoss()
        self.optimizer = optim.Adam(self.model.parameters(), lr=self.lr)
        self.train_loader, self.valid_loader = self.get_dataloaders(dataloaders['train_image_paths'], 
                                                                    dataloaders['train_labels'], 
                                                                    dataloaders['valid_image_paths'], 
                                                                    dataloaders['valid_labels'], 
                                                                    train_transforms=dataloaders['transforms'], 
                                                                    batch_size = self.batch_size,
                                                                    shuffle=parameter['shuffle'],
                                                                   sampler = dataloaders['sampler'])
        self.class_names = parameter['class_names']
        
        self.activations_path = os.path.join('activations', self.name)
        self.kernel_path = os.path.join('kernel_viz', self.name)
        save_path = os.path.join(os.getcwd(), 'models', self.name)
        if not os.path.exists(save_path):
            os.makedirs(save_path)

        if not os.path.exists(self.activations_path):
            os.makedirs(self.activations_path)

        if not os.path.exists(self.kernel_path):
            os.makedirs(self.kernel_path)
            
        self.save_path = save_path

    def train(self, save=True):
        '''
        @epochs: number of epochs to train
        @save: whether or not to save the checkpoints
        '''
        best_val_accuracy = - math.inf
        
        for epoch in range(self.epoch):  # loop over the dataset multiple times
            self.model.train()
            t = time.time()
            running_loss = 0.0
            train_acc = 0
            val_accuracy = 0
            correct = 0
            total = 0
            count = 0
            loop = tqdm.tqdm(self.train_loader, total = len(self.train_loader), leave = True)
            
            for img, label in loop:
                # get the inputs; data is a list of [inputs, labels]
                inputs, labels = img.to(device), label.to(device) #img.to(device), label.to(device)

                # zero the parameter gradients
                self.optimizer.zero_grad()

                # forward + backward + optimize
                outputs = self.model(inputs)
                _, predictions = torch.max(outputs, 1)
                loss = self.criterion(outputs, labels)
                loss.backward()
                self.optimizer.step()

                # print statistics
                running_loss += loss.item()
                total += labels.shape[0]
                correct += (predictions == labels).sum().item()
                
                count += 1
                if count % 2000 == 1999:    # print every 2000 mini-batches
                    print(f'[{epoch + 1}, {count + 1:5d}] loss: {running_loss / 2000:.3f}')
                    running_loss = 0.0
                    
            train_acc = 100 * correct / total
            print(f'Epoch:', epoch + 1, f'Training Epoch Accuracy:{train_acc}')
            
            # evaluate the validation dataset
            self.model.eval()
            correct_pred = {classname: 0 for classname in self.class_names}
            total_pred = {classname: 0 for classname in self.class_names}

            # again no gradients needed
            correct = 0
            total = 0
            with torch.no_grad():
                for data in self.valid_loader:
                    images, labels = data[0].to(device), data[1].to(device) #data[0], data[1]
                    outputs = self.model(images)
                    _, predictions = torch.max(outputs, 1)
                    # collect the correct predictions for each class
                    total += labels.shape[0]
                    correct += (predictions == labels).sum().item()

                    for label, prediction in zip(labels, predictions):
                        if label == prediction:
                            correct_pred[classes[label]] += 1
                        total_pred[classes[label]] += 1

            val_accuracy = 100 * correct / total
            print(f'Epoch:', epoch + 1, f'Validation Epoch Accuracy:{val_accuracy}')
                        
            # print the summary for each class
            print('Epoch:', epoch + 1, 'Correct predictions', correct_pred)
            print('Epoch:', epoch + 1, 'Total predictions', total_pred)
            print('Epoch:', epoch + 1, 'Correct predictions', correct_pred)
            print('Epoch:', epoch + 1, 'Total predictions', total_pred)
            
            # inspect the time taken to train one epoch
            d = time.time()-t
            print('Fininsh Trainig Epoch', epoch, '!', 'Time used:', d)
            
            if save:
                torch.save(self.model.state_dict(), os.path.join(self.save_path, f'epoch_{epoch}.pt'))
                if val_accuracy > best_val_accuracy:
                    torch.save(self.model.state_dict(), os.path.join(self.save_path, 'best.pt'))
                    best_val_accuracy = val_accuracy

        print('Done training!')                       

    
    def evaluate(self):
        # for evaluating the test dataset if there were any.
        try:
            assert os.path.exists(os.path.join(self.save_path, 'best.pt'))
            
        except:
            print('Please train first')
            return
        
        self.model.load_state_dict(torch.load(os.path.join(self.save_path, 'best.pt')))
        self.model.eval()
        
    def get_dataloaders(self, train_image_paths, train_labels, valid_image_paths, valid_labels, train_transforms=False, batch_size=32, shuffle=True, sampler = None):
        train_dataset = SurgicalDataset(train_image_paths,train_labels, train_transforms)
        val_dataset = SurgicalDataset(valid_image_paths,valid_labels, train_transforms)
        train_loader = DataLoader(train_dataset, batch_size, shuffle, sampler)
        valid_loader = DataLoader(val_dataset, batch_size, shuffle = True)
        return train_loader, valid_loader
    
    def grad_cam_on_input(self, img):
        
        try:
            assert os.path.exists(os.path.join(self.save_path, 'best.pt'))

        except:
            print('It appears you are testing the model without training. Please train first')
            return

        self.model.load_state_dict(torch.load(os.path.join(self.save_path, 'best.pt')))


        self.model.eval()
        img = img.to('cuda' if self.use_cuda else 'cpu')


        out = self.model(img)

        _, pred = torch.max(out, 1)

        predicted_class = self.class_names[int(pred)]
        print(f'Predicted class was {predicted_class}')

        out[:, pred].backward()
        gradients = self.model.get_gradient_activations()

        print('Gradients shape: ', f'{gradients.shape}')

        mean_gradients = torch.mean(gradients, [0, 2, 3]).cpu()
        activations = self.model.get_final_conv_layer(img).detach().cpu()

        print('Activations shape: ', f'{activations.shape}')

        for idx in range(activations.shape[1]):
            activations[:, idx, :, :] *= mean_gradients[idx]

        final_heatmap = np.maximum(torch.mean(activations, dim=1).squeeze(), 0)

        final_heatmap /= torch.max(final_heatmap)

        return final_heatmap

    def trained_kernel_viz(self):
        
        all_layers = [0, 3]
        all_filters = []
        for layer in all_layers:

            filters = self.model.conv_model[layer].weight
            all_filters.append(filters.detach().cpu().clone()[:8, :8, :, :])

        for filter_idx in range(len(all_filters)):

            filter = all_filters[filter_idx]
            print(filter.shape)
            filter = filter.contiguous().view(-1, 1, filter.shape[2], filter.shape[3])
            image = show_img(make_grid(filter))
            image = 255 * image
            cv2.imwrite(os.path.join(self.kernel_path, f'filter_layer{all_layers[filter_idx]}.jpg'), image)
    

    def activations_on_input(self, img):
        
        img = img.to('cuda' if self.use_cuda else 'cpu')

        all_layers = [0,3,6,8,10]
        all_viz = []
        
        # looking at the outputs of the relu
        for each in all_layers:

            current_model = self.model.conv_model[:each+1]
            current_out = current_model(img)
            all_viz.append(current_out.detach().cpu().clone()[:, :64, :, :])

        for viz_idx in range(len(all_viz)):

            viz = all_viz[viz_idx]
            viz = viz.view(-1, 1, viz.shape[2], viz.shape[3])
            image = show_img(make_grid(viz))
            image = 255 * image
            cv2.imwrite(os.path.join(self.activations_path, f'sample_layer{all_layers[viz_idx]}.jpg'), image)    
        

# Build and train models

In [9]:
example_model = models.alexnet(pretrained=True)
print(example_model)

AlexNet(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(11, 11), stride=(4, 4), padding=(2, 2))
    (1): ReLU(inplace=True)
    (2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(64, 192, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (4): ReLU(inplace=True)
    (5): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Conv2d(192, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): ReLU(inplace=True)
    (8): Conv2d(384, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): ReLU(inplace=True)
    (10): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (avgpool): AdaptiveAvgPool2d(output_size=(6, 6))
  (classifier): Sequential(
    (0): Dropout(p=0.5, inplace=False)
    (1): Linear(in_features=9216, out_features=4096, bias=True)
 

In [10]:
class TransferAlexNet(nn.Module):
    def __init__(self):
        super().__init__()
        self.base_alex_net = models.alexnet(pretrained=True)
        self.conv_model = self.get_conv_layers()
        self.final_max_pool = self.final_pool_layer()
        self.avg_pool = self.transition_layer()
        self.fc_model = self.get_fc_layers()
        self.activate_training_layers()

    def activate_training_layers(self):
        for name, param in self.conv_model.named_parameters():
            number = int(name.split('.')[0])
            # for all layers except the last layer set param.requires_grad = False
            if number < 10:
                param.requires_grad = False
            else:
                param.requires_grad = True
                
        for name, param in self.fc_model.named_parameters():
            # for all of these layers set param.requires_grad as True
            param.requires_grad = True

    def get_conv_layers(self):
        return self.base_alex_net.features[:12]

    def final_pool_layer(self):
        return nn.MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)

    def transition_layer(self):
        return nn.AdaptiveAvgPool2d(output_size=(6, 6))

    def get_fc_layers(self):
        return nn.Sequential(
            nn.Dropout(p=0.5, inplace=False),
            nn.Linear(in_features=9216, out_features=4096, bias=True),
            nn.ReLU(inplace=True),
            nn.Dropout(p=0.5, inplace=False),
            nn.Linear(in_features=4096, out_features=4096, bias=True),
            nn.ReLU(inplace=True),
            nn.Linear(in_features=4096, out_features=1000, bias=True),
            nn.ReLU(inplace=True),
            nn.Linear(in_features=1000, out_features=15, bias=True),
        )
    
    def forward(self, x):
        x = self.conv_model(x)   #call the conv layers
        x = self.final_max_pool(x)  #call the max pool layer
        x = self.avg_pool(x)  #call the avg pool layer
        x = torch.flatten(x, 1)
        x = self.fc_model(x)  #call fully connected layers  
        
        return x

In [11]:
# Train a transfer learning model with Alexnet
name = 'TransferAlexNet'
classes = [i for i in range(15)]
transforms = get_transform('alexnet')
dataloaders = {'train_image_paths': train_image_paths, 'train_labels' : train_labels, 'valid_image_paths': valid_image_paths, 'valid_labels':valid_labels, 'transforms':transforms, 'sampler':sampler}
parameters = {'lr': 0.001, 'epochs' : 5, 'batch_size':32, 'shuffle':False, 'class_names':classes}

model = TransferAlexNet()
classifier = Classifier(name, model, dataloaders, parameters, use_cuda=True)
classifier.train()

 37%|██████████████████████████████████▌                                                          | 1999/5377 [07:26<13:20,  4.22it/s]

[1,  2000] loss: 0.761


 74%|█████████████████████████████████████████████████████████████████████▏                       | 3999/5377 [14:48<05:03,  4.55it/s]

[1,  4000] loss: 0.469


100%|█████████████████████████████████████████████████████████████████████████████████████████████| 5377/5377 [19:53<00:00,  4.51it/s]


...

[5,  2000] loss: 0.238


 74%|█████████████████████████████████████████████████████████████████████▏                       | 3999/5377 [15:27<05:14,  4.38it/s]

[5,  4000] loss: 0.240


100%|█████████████████████████████████████████████████████████████████████████████████████████████| 5377/5377 [20:47<00:00,  4.31it/s]


Epoch: 5 Training Epoch Accuracy:93.04309918916563
Epoch: 5 Validation Epoch Accuracy:89.69822375151121
Epoch: 5 Correct predictions {0: 49, 1: 2117, 2: 5491, 3: 9229, 4: 163, 5: 5288, 6: 136, 7: 2370, 8: 243, 9: 561, 10: 10284, 11: 1953, 12: 399, 13: 281, 14: 17}
Epoch: 5 Total predictions {0: 52, 1: 2177, 2: 5799, 3: 10405, 4: 213, 5: 5470, 6: 158, 7: 2843, 8: 247, 9: 587, 10: 11135, 11: 3216, 12: 404, 13: 288, 14: 18}
Epoch: 5 Correct predictions {0: 49, 1: 2117, 2: 5491, 3: 9229, 4: 163, 5: 5288, 6: 136, 7: 2370, 8: 243, 9: 561, 10: 10284, 11: 1953, 12: 399, 13: 281, 14: 17}
Epoch: 5 Total predictions {0: 52, 1: 2177, 2: 5799, 3: 10405, 4: 213, 5: 5470, 6: 158, 7: 2843, 8: 247, 9: 587, 10: 11135, 11: 3216, 12: 404, 13: 288, 14: 18}
Fininsh Trainig Epoch 4 ! Time used: 1544.173043012619
Done training!


# Data Augmentations for scarse data

In [ ]:
# find out the ratio of different labels/data

# we decide to augment the scarser data in the following scheme: label 0, + 400, label 14 + 500, label 6 + 200


In [65]:
candidates = [2,3,6,7]
target = 8
# Output： [[7], [2,2,3]]

In [66]:
# wrong!

def recur(candidate, target, candidates):
    #print(candidate)
    if sum(candidate) > target:
        return []
    if sum(candidate) == target:
        return candidate
    
    combination = []
    candidate = candidate[:]
    for num in candidates:
        candidate.append(num)
        combination.extend(recur(candidate, target, candidates))
        
    print(combination)  
    return combination
    
output = []
for i in candidates:
    candidate = []
    candidate.append(i)
    combination = recur(candidate, target, candidates)
    if combination: output.append(combination)

print(output)
    
        
    

[2, 2, 2, 2]
[2, 2, 2, 2]
[]
[2, 2, 2, 2]
[]
[]
[3, 2, 3]
[6, 2]
[]
[[2, 2, 2, 2], [3, 2, 3], [6, 2]]


In [67]:
def findcomb(candidates, target):
    res = []
    def dfs(acc, i):
        acc.append(candidates[i])
        print(acc)
        if sum(acc) == target:
            res.append(acc[:])
            return
        if sum(acc) > target:
            return
        for j in range(i, len(candidates)):
            #acc.append(candidates[j])
            dfs(acc, j)
            acc.pop()
    
    for i in range(len(candidates)):
        acc = []
        dfs(acc, i)
    return res

findcomb(candidates, target)

[2]
[2, 2]
[2, 2, 2]
[2, 2, 2, 2]
[2, 2, 2, 3]
[2, 2, 2, 6]
[2, 2, 2, 7]
[2, 2, 3]
[2, 2, 3, 3]
[2, 2, 3, 6]
[2, 2, 3, 7]
[2, 2, 6]
[2, 2, 7]
[2, 3]
[2, 3, 3]
[2, 3, 6]
[2, 3, 7]
[2, 6]
[2, 7]
[3]
[3, 3]
[3, 3, 3]
[3, 3, 6]
[3, 3, 7]
[3, 6]
[3, 7]
[6]
[6, 6]
[6, 7]
[7]
[7, 7]


[[2, 2, 2, 2], [2, 3, 3], [2, 6]]